In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt 


from sklearn.linear_model import LinearRegression

from sklearn import preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import xgboost
from xgboost import XGBClassifier
from xgboost import XGBRegressor



def rmse(x,y):
    rmse = np.sqrt(sklearn.metrics.mean_squared_error(x,y))
    return(rmse)

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv',index_col = 'id')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv',index_col = 'id')

In [ ]:
train.head()

In [ ]:
# convert all the categorical variable to multiclass
le = preprocessing.LabelEncoder()
le.fit(train['cat9'])

cat =  ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8','cat9']

for i in cat:
    train[i] = le.transform(train[i].values)
    
for i in cat:
    test[i] = le.transform(test[i].values)


In [ ]:
train.head()

In [ ]:
# prepare datasets
X = train.drop(columns=['target'])

y = train.iloc[:,-1]

In [ ]:
#split among train and validation
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25, random_state=42)


## xgboost

In [ ]:
# create a dictionary with the parameter used for tuning
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5,1.5, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8],
        'learning_rate' : [0.02,0.1]
        }

In [ ]:
# initialize a regression random forest, specifying the number of trees, the objective function and the gpu
xgb = XGBRegressor(n_estimators=800, objective='reg:squarederror',tree_method='gpu_hist')


In [ ]:
# initialize the gridsearch specifying the random forest create before and the dictionary with the parameters
clf = GridSearchCV(xgb,params, 
                    verbose=1)

In [ ]:
# normal fit procedure, specifying the validation set
clf.fit(X_train,y_train,eval_set=[(X_val,y_val)])


In [ ]:
# using the specified (root mean square error)metric in order to evaluate the model
rmse(clf.predict(X),y)

In [ ]:
#prediction = clf.predict(test)

## submission

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv',index_col='id')

In [ ]:
submission['target'] = prediction

In [ ]:
submission.to_csv('submission.csv')